In [ ]:
import pandas as pds
import numpy as np
import os
from IPython.display import display

In [ ]:
# Read and join tables

leaderboards = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\leaderboards-data.csv"), parse_dates=['date', 'verifiedDate'])
games = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\games-data.csv"), parse_dates=['releaseDate', 'createdDate'])
users = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\users-data.csv"), parse_dates=['signupDate'])
categories = pds.read_csv(os.path.join(os.getcwd(),"data\\v1\\categories-data.csv"))

data_frame = leaderboards.merge(games, left_on="gameID", right_on="ID", how="left", suffixes=('_df1', '_gme'))
data_frame = data_frame.merge(users, left_on="players", right_on="ID", how="left", suffixes=('_df2', '_usr'))
data_frame = data_frame.merge(categories, left_on="categoryID", right_on="ID", how="left", suffixes=('_df3', '_cat'))
target = data_frame['emulated']
data_frame = data_frame.drop(columns=['runID', 'ID_df2', 'name_df2', 'URL', 'name_usr', 'parentGameID', 'ID', 'name', 'rules'])
 
# Split players rows where there are multiple players into multiple rows 
cols = list(data_frame.columns.drop('players' ))
data_frame = (data_frame
   .set_index(cols)
   .stack()
   .str.split(',', expand=True)
   .stack()
   .unstack(-2)
   .reset_index(-1, drop=True)
   .reset_index()
)



In [ ]:
# Print Table

pds.set_option('display.max_columns', None)
#data_frame = data_frame.rename(columns={})
display(data_frame)

In [ ]:
# Preprocessing

uniqueLocations = data_frame['location'].unique()
locationMappings = dict(zip(uniqueLocations, range(len(uniqueLocations))))
data_frame = data_frame.replace({'location': locationMappings})

#TODO: Replace NAs before converting to unix timestamp
data_frame['date'] = data_frame['date'].astype('int64') // (10**9)
data_frame['verifiedDate'] = data_frame['verifiedDate'].astype('int64') // (10**9)
data_frame['releaseDate'] = data_frame['releaseDate'].astype('int64') // (10**9)
data_frame['createdDate'] = data_frame['createdDate'].astype('int64') // (10**9)
data_frame['signupDate'] = data_frame['signupDate'].astype('int64') // (10**9)
display(data_frame)